<a href="https://colab.research.google.com/github/qahaidari/Deep-learning-for-classification-and-regression/blob/master/test_for_Aalto_course.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
pwd

'/content'

In [5]:
import zipfile
with zipfile.ZipFile('/content/lsp_dataset.zip', 'r') as zip_ref:
  zip_ref.extractall('/content/lsp_dataset')

In [ ]:
# 1. Data loading for regression: Using custom dataset for LSP
# Link for downloading LSP dataset: http://sam.johnson.io/research/lsp.html 
# Visualization without resizing images
import torch
import torchvision
from torchvision import transforms, utils
import torchvision.datasets as datasets
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import scipy.io as sio
from os.path import dirname, normpath, normcase, join as pjoin
import numpy as np
import urllib.request
from skimage import io
import os

class CustomLSP(Dataset):
  
  def __init__(self, data_dir, root_dir):
    joints_dir = pjoin(data_dir, 'joints.mat')
    self.images_folder = pjoin(data_dir, 'images')
    mat_contents = sio.loadmat(joints_dir) # labels are in the form of MATLAB mat files
    self.joints = mat_contents['joints']
    self.images = sorted(os.listdir(self.images_folder)) # sorted list of images
    
  def __getitem__(self, index):
    im = pjoin(self.images_folder, self.images[index])
    im = plt.imread(im) # numpy array of the image
    joints = self.joints[:,:,index]
    return im, joints
  
  def __len__(self):
    self.len = print(im.shape)
    return self.len

data_dir = pjoin('./lsp_dataset') # LSP dataset should already exist in the main root of the project directory
data = CustomLSP(data_dir, root_dir='/')

################### Visualization ############################
num_of_images = 3
for index in range(0, num_of_images):
  images, joints = data[index]
  print('sample #{} has shape {}'.format(index, images.shape))
  f = plt.figure(figsize=(8,8))
  ax = f.add_subplot(1, num_of_images, index+1)
  ax.axis('off')
  ax.imshow(images)
  ax.plot([joints[0,0],joints[0,1],joints[0,2]],[joints[1,0],joints[1,1],joints[1,2]],marker = 'o', c='r', zorder=1)
  ax.plot([joints[0,3],joints[0,4],joints[0,5]],[joints[1,3],joints[1,4],joints[1,5]],marker = 'o', c='r', zorder=1)
  ax.plot([joints[0,6],joints[0,7],joints[0,8]],[joints[1,6],joints[1,7],joints[1,8]],marker = 'o', c='r', zorder=1)
  ax.plot([joints[0,9],joints[0,10],joints[0,11]],[joints[1,9],joints[1,10],joints[1,11]],marker = 'o', c='r', zorder=1)
  ax.plot([joints[0,12],joints[0,13]],[joints[1,12],joints[1,13]],marker = 'o', c='r', zorder=1)

In [ ]:
# 2. Data loading for regression: Using custom data loader for LSP with variable batch size dataloader and splitting dataset to train and test sets
# Link for downloading LSP dataset: http://sam.johnson.io/research/lsp.html
# Visualization after resizing images
import torch
import torchvision
from torchvision import transforms, utils
import torchvision.datasets as datasets
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import scipy.io as sio
from os.path import dirname, normpath, normcase, join as pjoin
import numpy as np
import urllib.request
from skimage import io
import os
import math
import cv2

Batch_size = 3; # samples in one batch
train_size = 80; # 80% of the data
test_size = 100 - train_size; # 20% of the data 

class CustomLSP(Dataset):
  
  def __init__(self, data_dir, train_size, train):
    joints_dir = pjoin(data_dir, 'joints.mat')
    images_folder = pjoin(data_dir, 'images')
    mat_contents = sio.loadmat(joints_dir)
    joints = mat_contents['joints']
    images = sorted(os.listdir(images_folder)) # sorted list of images
    self.train = train

    if self.train == True:
      self.train_len = train_size * len(images) / 100 # this is train_size * 2000 / 100
      self.train_image_set = []
      self.train_joint_set = []
      for i in range(0,int(self.train_len)):
        
        # resizing images to equal sizes and modifying joints accordingly
        im = plt.imread(pjoin(images_folder,images[i]))
        h, w = im.shape[:2]
        im = cv2.resize(im, (256, 256)) 
        im = torch.from_numpy(im)
        self.train_image_set.append(im)
        joints_i = torch.from_numpy(joints[:2,:,i]) # ignoring the visibility term in joints
        joints_i[0,:] = torch.mul(joints_i[0,:], 256/w) # modifying joints
        joints_i[1,:] = torch.mul(joints_i[1,:], 256/h) # modifying joints
        self.train_joint_set.append(joints_i)
        
    if self.train == False:
      self.train_len = train_size * len(images) / 100 # this is train_size * 2000 / 100
      self.test_len = (100 - train_size) * len(images) / 100
      self.test_image_set = []
      self.test_joint_set = []
      for i in range(int(self.train_len),len(images)):

        # resizing images to equal sizes and modifying joints accordingly
        im = plt.imread(pjoin(images_folder,images[i]))
        h , w = im.shape[:2]
        im = cv2.resize(im, (256, 256)) 
        im = torch.from_numpy(im)
        self.test_image_set.append(im)
        joints_i = torch.from_numpy(joints[:2,:,i]) # ignoring the visibility term in joints
        joints_i[0,:] = torch.mul(joints_i[0,:], 256/w) # modifying joints
        joints_i[1,:] = torch.mul(joints_i[1,:], 256/h) # modifying joints
        self.test_joint_set.append(joints_i)

  def __getitem__(self, index):
    if self.train == True:
      train_images = self.train_image_set[index]
      train_joints = self.train_joint_set[index]
      return train_images, train_joints
    if self.train == False:
      test_images = self.test_image_set[index]
      test_joints = self.test_joint_set[index]
      return test_images, test_joints
  
  def __len__(self):
    if self.train == True:
      return int(self.train_len)
    if self.train == False:
      return int(self.test_len)

data_dir = pjoin('./lsp_dataset') # LSP dataset should already exist in the main root of the project directory
train_set = CustomLSP(data_dir, train_size=80, train=True)
test_set = CustomLSP(data_dir, train_size=80, train=False)
train_loader = torch.utils.data.DataLoader(train_set, Batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_set, Batch_size, shuffle=True)
dataiter = iter(train_loader)
train_images, train_joints = dataiter.next()
################### Visualization ############################
num_of_images = Batch_size
for index in range(0, num_of_images):
  image = train_images[index]
  joints = train_joints[index]
  print('sample #{} has shape {}'.format(index, image.shape))
  f = plt.figure(figsize=(10,10))
  ax = f.add_subplot(1, num_of_images, index+1)
  ax.axis('off')
  ax.imshow(image)
  ax.plot([joints[0,0],joints[0,1],joints[0,2]],[joints[1,0],joints[1,1],joints[1,2]],marker = 'o', c='r', zorder=1)
  ax.plot([joints[0,3],joints[0,4],joints[0,5]],[joints[1,3],joints[1,4],joints[1,5]],marker = 'o', c='r', zorder=1)
  ax.plot([joints[0,6],joints[0,7],joints[0,8]],[joints[1,6],joints[1,7],joints[1,8]],marker = 'o', c='r', zorder=1)
  ax.plot([joints[0,9],joints[0,10],joints[0,11]],[joints[1,9],joints[1,10],joints[1,11]],marker = 'o', c='r', zorder=1)
  ax.plot([joints[0,12],joints[0,13]],[joints[1,12],joints[1,13]],marker = 'o', c='r', zorder=1)

In [ ]:
# 3. Model Training for Regression: LSP
# Link for downloading LSP dataset: http://sam.johnson.io/research/lsp.html
import torch
from torch import nn
import torchvision
from torchvision import transforms, utils
import torchvision.datasets as datasets
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import scipy.io as sio
from os.path import dirname, normpath, normcase, join as pjoin
import numpy as np
import urllib.request
from skimage import io
import os
import math
from torch.nn.functional import cross_entropy
import pickle
import cv2

class Network(nn.Module):
    def __init__(self, input_shape):
        super(Network, self).__init__()

        self.conv1 = nn.Conv2d(input_shape[0], 6, (5, 5))
        self.act1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(2, stride=2)
        self.conv2 = nn.Conv2d(6, 12, (5, 5))
        self.act2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(2, stride=2)
        self.conv3 = nn.Conv2d(12, 12, (5, 5))
        self.act3 = nn.ReLU()
        self.pool3 = nn.MaxPool2d(2, stride=1)
        self.linear1 = nn.Linear(56*56*12, 120)
        self.act4 = nn.ReLU()
        self.linear2 = nn.Linear(120, 60)
        self.act5 = nn.ReLU()
        self.linear3 = nn.Linear(60,30) 
        self.act6 = nn.ReLU()
        self.linear4 = nn.Linear(30,28)

    def forward(self, x):
        x = self.pool1(self.act1(self.conv1(x.float())))
        x = self.pool2(self.act2(self.conv2(x)))
        x = self.pool3(self.act3(self.conv3(x)))
        x = x.view(-1, 56*56*12)
        x = self.act4(self.linear1(x))
        x = self.act5(self.linear2(x))
        x = self.act6(self.linear3(x))
        x = self.linear4(x)
        return x

class CustomLSP(Dataset):
  
  def __init__(self, data_dir, train_size, train):
    joints_dir = pjoin(data_dir, 'joints.mat')
    images_folder = pjoin(data_dir, 'images')
    mat_contents = sio.loadmat(joints_dir)
    joints = mat_contents['joints']
    images = sorted(os.listdir(images_folder)) # sorted list of images
    self.train = train

    if self.train == True:
      self.train_len = train_size * len(images) / 100 # this is train_size * 2000 / 100
      self.train_image_set = []
      self.train_joint_set = []
      for i in range(0,int(self.train_len)):
        
        # resizing images to equal sizes and modifying joints accordingly
        im = plt.imread(pjoin(images_folder,images[i]))
        h, w = im.shape[:2]
        im = cv2.resize(im, (256, 256)) 
        im = torch.from_numpy(im)

        im = im.view(3,1,256*256).view(3,256,256)

        self.train_image_set.append(im)
        joints_i = torch.from_numpy(joints[:2,:,i]) # ignoring the visibility term in joints
        joints_i[0,:] = torch.mul(joints_i[0,:], 256/w) # modifying joints
        joints_i[1,:] = torch.mul(joints_i[1,:], 256/h) # modifying joints
        joints_i = torch.reshape(joints_i,(1,28)) # joints reshaped so that later joints and predictions have same shape for loss calculation
        self.train_joint_set.append(joints_i)  
        
    if self.train == False:
      self.train_len = train_size * len(images) / 100 # this is train_size * 2000 / 100
      self.test_len = (100 - train_size) * len(images) / 100
      self.test_image_set = []
      self.test_joint_set = []
      for i in range(int(self.train_len),len(images)):

        # resizing images to equal sizes and modifying joints accordingly
        im = plt.imread(pjoin(images_folder,images[i]))
        h, w = im.shape[:2]
        im = cv2.resize(im, (256, 256)) 
        im = torch.from_numpy(im)
        
        im = im.view(3,1,256*256).view(3,256,256)

        self.test_image_set.append(im)
        joints_i = torch.from_numpy(joints[:2,:,i]) # ignoring the visibility term in joints
        joints_i[0,:] = torch.mul(joints_i[0,:], 256/w) # modifying joints
        joints_i[1,:] = torch.mul(joints_i[1,:], 256/h) # modifying joints
        joints_i = torch.reshape(joints_i,(1,28)) # joints reshaped so that later joints and predictions have same shape for loss calculation
        self.test_joint_set.append(joints_i)
        
  def __getitem__(self, index):
    if self.train == True:
      train_images = self.train_image_set[index]
      train_joints = self.train_joint_set[index]
      return train_images, train_joints
    if self.train == False:
      test_images = self.test_image_set[index]
      test_joints = self.test_joint_set[index]
      return test_images, test_joints
  
  def __len__(self):
    if self.train == True:
      return int(self.train_len)
    if self.train == False:
      return int(self.test_len)

NUM_EPOCHS = 10 # number of iterations
BATCH_SIZE = 64 # samples in one batch
LR = 0.001 # learning rate
train_size = 80 # 80% of the data
test_size = 100 - train_size # 20% of the data 

def main():
  data_dir = pjoin('./lsp_dataset') # LSP dataset already exists in the main root of the project directory
  train_set = CustomLSP(data_dir, train_size=80, train=True)
  test_set = CustomLSP(data_dir, train_size=80, train=False)
  train_loader = torch.utils.data.DataLoader(train_set, BATCH_SIZE, shuffle=True)
  test_loader = torch.utils.data.DataLoader(test_set, BATCH_SIZE, shuffle=True)

  # instantiate the model
  model = Network((3,256, 256))
  optimizer = torch.optim.Adam(model.parameters(), lr=LR)

  train_losses = []
  test_losses = []
  for epoch in range(NUM_EPOCHS):
    for mode, data in [("train", train_loader), ("test", test_loader)]:
  
      runningLoss = 0.
      correct = 0
      total = 0
      for step, (images, joints) in enumerate(data): # looping over each batch data
        
        predictions = model.forward(images)
        total += images.shape[0]

        error = joints - predictions
        loss = error.pow(2).mean()
    
        runningLoss += loss.item() * images.shape[0]
        
        if mode == "train":
          optimizer.zero_grad()
          loss.backward()
          optimizer.step()

      (train_losses if mode == "train" else test_losses).append(runningLoss / total)
      print("epochs: {} | {} | total loss: {}".format(epoch,mode,runningLoss/total))

  with open('/LSP_train_test_losses.pickle', "wb") as f: # file = 'LSP_train_test_losses.pickle'
    pickle.dump((train_losses, test_losses), f)
          
if __name__ == "__main__":
    main()

######################## Plots #################################################
with open("/LSP_train_test_losses.pickle", "rb") as f:
        train_losses, test_losses = pickle.load(f)

plt.plot(np.arange(0, len(train_losses)), train_losses, label= "train loss", color="blue")
plt.plot(np.arange(0, len(train_losses)), test_losses, label= "test loss", color="green")
plt.xlabel("epochs")
plt.ylabel("loss")
plt.legend()
plt.show()